In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [127]:
weather_data = ['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
years = ["2016","2017"]
weather = ['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph','tempF','WindChillF','humidity','date','time', 'airport']

weatherdf = pd.DataFrame()

for airport in weather_data:
    for year in years:
        for i in range(1,13):
            with open("weather/"+airport+"/"+year+"-"+str(i)+".json", 'r') as f:
                data = json.load(f)
            a = json_normalize(data['data']['weather'], 'hourly', ['date'] )
            b = json_normalize(data['data']['request'])
            a['airport'] = str(b['query'])[5:8]
            a = a.filter(weather)
            weatherdf = pd.concat([weatherdf,a])


,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL
5,17,315,122,0.0,10,1024,100,38,29,40,33,93,2016-01-01,500,ATL
6,18,315,122,0.0,10,1024,100,38,28,40,32,93,2016-01-01,600,ATL
7,18,317,122,0.0,10,1024,100,38,26,41,33,93,2016-01-01,700,ATL
8,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,800,ATL
9,17,319,122,0.0,10,1025,100,39,20,42,35,92,2016-01-01,900,ATL


In [131]:
def timetofloat(r):
  return int(r['time'])*1.0

weatherdf["time"] = weatherdf.apply(lambda r: timetofloat(r), axis = 1)

def unique(r):
  return r["airport"]+"||"+r["date"]+"||"+str(r["time"])

weatherdf["key"] = weatherdf.apply(lambda r: unique(r), axis = 1)
weatherdf.index = weatherdf["key"]
del weatherdf['key']

weatherdf

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
key,,,,,,,,,,,,,,,
ATL||2016-01-01||0.0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0.0,ATL
ATL||2016-01-01||100.0,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100.0,ATL
ATL||2016-01-01||200.0,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200.0,ATL
ATL||2016-01-01||300.0,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300.0,ATL
ATL||2016-01-01||400.0,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400.0,ATL
ATL||2016-01-01||500.0,17,315,122,0.0,10,1024,100,38,29,40,33,93,2016-01-01,500.0,ATL
ATL||2016-01-01||600.0,18,315,122,0.0,10,1024,100,38,28,40,32,93,2016-01-01,600.0,ATL
ATL||2016-01-01||700.0,18,317,122,0.0,10,1024,100,38,26,41,33,93,2016-01-01,700.0,ATL
ATL||2016-01-01||800.0,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,800.0,ATL


In [137]:
flightdf = pd.DataFrame()

for year in years:
    for i in range(1,13):
        df = pd.read_csv('Flight-Data/'+year+'_original/'+year+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'.csv', engine = 'python')
        flight = ['FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Origin']
        df = df.filter(flight)
        df['DepTime'] *= 0.01
        df['DepTime'] = df['DepTime'].apply(np.floor)
        df['DepTime'] *= 100
        flightdf = pd.concat([flightdf,df])
        
flightdf
        

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin
0,2016-01-06,1,2016,1,6,1000.0,0.0,1100,0.0,11298,11433,1432.0,1438,0.0,0.0,DFW
1,2016-01-07,1,2016,1,7,1000.0,0.0,1100,0.0,11298,11433,1426.0,1438,0.0,0.0,DFW
2,2016-01-08,1,2016,1,8,1000.0,0.0,1100,0.0,11298,11433,1445.0,1438,0.0,7.0,DFW
3,2016-01-09,1,2016,1,9,1100.0,0.0,1100,2.0,11298,11433,1433.0,1438,0.0,0.0,DFW
4,2016-01-10,1,2016,1,10,1200.0,1.0,1100,100.0,11298,11433,1631.0,1438,1.0,113.0,DFW
5,2016-01-11,1,2016,1,11,1100.0,0.0,1100,7.0,11298,11433,1435.0,1438,0.0,0.0,DFW
6,2016-01-12,1,2016,1,12,1000.0,0.0,1100,0.0,11298,11433,1438.0,1438,0.0,0.0,DFW
7,2016-01-13,1,2016,1,13,1000.0,0.0,1100,0.0,11298,11433,1431.0,1438,0.0,0.0,DFW
8,2016-01-14,1,2016,1,14,1000.0,0.0,1100,0.0,11298,11433,1428.0,1438,0.0,0.0,DFW
9,2016-01-15,1,2016,1,15,1000.0,0.0,1100,0.0,11298,11433,1434.0,1438,0.0,0.0,DFW


In [138]:
def unique(r):
  return r["Origin"]+"||"+r["FlightDate"]+"||"+str(r["DepTime"])

flightdf["key"] = flightdf.apply(lambda r: unique(r), axis = 1)

flightdf.index = flightdf["key"]
del flightdf['key']

flightdf

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin
key,,,,,,,,,,,,,,,,
DFW||2016-01-06||1000.0,2016-01-06,1,2016,1,6,1000.0,0.0,1100,0.0,11298,11433,1432.0,1438,0.0,0.0,DFW
DFW||2016-01-07||1000.0,2016-01-07,1,2016,1,7,1000.0,0.0,1100,0.0,11298,11433,1426.0,1438,0.0,0.0,DFW
DFW||2016-01-08||1000.0,2016-01-08,1,2016,1,8,1000.0,0.0,1100,0.0,11298,11433,1445.0,1438,0.0,7.0,DFW
DFW||2016-01-09||1100.0,2016-01-09,1,2016,1,9,1100.0,0.0,1100,2.0,11298,11433,1433.0,1438,0.0,0.0,DFW
DFW||2016-01-10||1200.0,2016-01-10,1,2016,1,10,1200.0,1.0,1100,100.0,11298,11433,1631.0,1438,1.0,113.0,DFW
DFW||2016-01-11||1100.0,2016-01-11,1,2016,1,11,1100.0,0.0,1100,7.0,11298,11433,1435.0,1438,0.0,0.0,DFW
DFW||2016-01-12||1000.0,2016-01-12,1,2016,1,12,1000.0,0.0,1100,0.0,11298,11433,1438.0,1438,0.0,0.0,DFW
DFW||2016-01-13||1000.0,2016-01-13,1,2016,1,13,1000.0,0.0,1100,0.0,11298,11433,1431.0,1438,0.0,0.0,DFW
DFW||2016-01-14||1000.0,2016-01-14,1,2016,1,14,1000.0,0.0,1100,0.0,11298,11433,1428.0,1438,0.0,0.0,DFW


In [146]:

def check(r):
    if r["Origin"] not in weather_data:
        return 1
    else:
        return 0

flightdf["y"] = flightdf.apply(lambda r: check(r), axis = 1)

flightdf = flightdf[flightdf["y"]!=1]

flightdf


,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,y
key,,,,,,,,,,,,,,,,,
DFW||2016-01-06||1000.0,2016-01-06,1,2016,1,6,1000.0,0.0,1100,0.0,11298,11433,1432.0,1438,0.0,0.0,DFW,0
DFW||2016-01-07||1000.0,2016-01-07,1,2016,1,7,1000.0,0.0,1100,0.0,11298,11433,1426.0,1438,0.0,0.0,DFW,0
DFW||2016-01-08||1000.0,2016-01-08,1,2016,1,8,1000.0,0.0,1100,0.0,11298,11433,1445.0,1438,0.0,7.0,DFW,0
DFW||2016-01-09||1100.0,2016-01-09,1,2016,1,9,1100.0,0.0,1100,2.0,11298,11433,1433.0,1438,0.0,0.0,DFW,0
DFW||2016-01-10||1200.0,2016-01-10,1,2016,1,10,1200.0,1.0,1100,100.0,11298,11433,1631.0,1438,1.0,113.0,DFW,0
DFW||2016-01-11||1100.0,2016-01-11,1,2016,1,11,1100.0,0.0,1100,7.0,11298,11433,1435.0,1438,0.0,0.0,DFW,0
DFW||2016-01-12||1000.0,2016-01-12,1,2016,1,12,1000.0,0.0,1100,0.0,11298,11433,1438.0,1438,0.0,0.0,DFW,0
DFW||2016-01-13||1000.0,2016-01-13,1,2016,1,13,1000.0,0.0,1100,0.0,11298,11433,1431.0,1438,0.0,0.0,DFW,0
DFW||2016-01-14||1000.0,2016-01-14,1,2016,1,14,1000.0,0.0,1100,0.0,11298,11433,1428.0,1438,0.0,0.0,DFW,0


In [147]:
final = pd.concat([flightdf, weatherdf], axis=1, join_axes=[flightdf.index])
final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
key,,,,,,,,,,,,,,,,,,,,,
DFW||2016-01-06||1000.0,2016-01-06,1,2016,1,6,1000.0,0.0,1100,0.0,11298,...,1023,100,39,15,44,37,88,2016-01-06,1000.0,DFW
DFW||2016-01-07||1000.0,2016-01-07,1,2016,1,7,1000.0,0.0,1100,0.0,11298,...,1012,78,46,19,52,50,92,2016-01-07,1000.0,DFW
DFW||2016-01-08||1000.0,2016-01-08,1,2016,1,8,1000.0,0.0,1100,0.0,11298,...,1010,89,43,12,50,50,94,2016-01-08,1000.0,DFW
DFW||2016-01-09||1100.0,2016-01-09,1,2016,1,9,1100.0,0.0,1100,2.0,11298,...,1013,100,37,36,42,33,82,2016-01-09,1100.0,DFW
DFW||2016-01-10||1200.0,2016-01-10,1,2016,1,10,1200.0,1.0,1100,100.0,11298,...,1029,0,23,16,35,27,66,2016-01-10,1200.0,DFW
DFW||2016-01-11||1100.0,2016-01-11,1,2016,1,11,1100.0,0.0,1100,7.0,11298,...,1026,93,27,10,41,37,73,2016-01-11,1100.0,DFW
DFW||2016-01-12||1000.0,2016-01-12,1,2016,1,12,1000.0,0.0,1100,0.0,11298,...,1025,0,34,9,44,43,87,2016-01-12,1000.0,DFW
DFW||2016-01-13||1000.0,2016-01-13,1,2016,1,13,1000.0,0.0,1100,0.0,11298,...,1023,3,37,17,47,46,86,2016-01-13,1000.0,DFW
DFW||2016-01-14||1000.0,2016-01-14,1,2016,1,14,1000.0,0.0,1100,0.0,11298,...,1015,36,45,28,51,49,89,2016-01-14,1000.0,DFW


In [154]:
final.reset_index()

del final['y']
del final['date']
del final['Origin']

final.dtypes

FlightDate          object
Quarter              int64
Year                 int64
Month                int64
DayofMonth           int64
DepTime            float64
DepDel15           float64
CRSDepTime           int64
DepDelayMinutes    float64
OriginAirportID      int64
DestAirportID        int64
ArrTime            float64
CRSArrTime           int64
ArrDel15           float64
ArrDelayMinutes    float64
Origin              object
windspeedKmph       object
winddirDegree       object
weatherCode         object
precipMM            object
visibility          object
pressure            object
cloudcover          object
DewPointF           object
WindGustKmph        object
tempF               object
WindChillF          object
humidity            object
time               float64
airport             object
dtype: object

In [161]:
final.reset_index()

final.index = range(len(final))

final

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,airport
0,2016-01-06,1,2016,1,6,1000.0,0.0,1100,0.0,11298,...,6,1023,100,39,15,44,37,88,1000.0,DFW
1,2016-01-07,1,2016,1,7,1000.0,0.0,1100,0.0,11298,...,2,1012,78,46,19,52,50,92,1000.0,DFW
2,2016-01-08,1,2016,1,8,1000.0,0.0,1100,0.0,11298,...,2,1010,89,43,12,50,50,94,1000.0,DFW
3,2016-01-09,1,2016,1,9,1100.0,0.0,1100,2.0,11298,...,8,1013,100,37,36,42,33,82,1100.0,DFW
4,2016-01-10,1,2016,1,10,1200.0,1.0,1100,100.0,11298,...,10,1029,0,23,16,35,27,66,1200.0,DFW
5,2016-01-11,1,2016,1,11,1100.0,0.0,1100,7.0,11298,...,10,1026,93,27,10,41,37,73,1100.0,DFW
6,2016-01-12,1,2016,1,12,1000.0,0.0,1100,0.0,11298,...,6,1025,0,34,9,44,43,87,1000.0,DFW
7,2016-01-13,1,2016,1,13,1000.0,0.0,1100,0.0,11298,...,6,1023,3,37,17,47,46,86,1000.0,DFW
8,2016-01-14,1,2016,1,14,1000.0,0.0,1100,0.0,11298,...,10,1015,36,45,28,51,49,89,1000.0,DFW
9,2016-01-15,1,2016,1,15,1000.0,0.0,1100,0.0,11298,...,6,1010,5,38,21,49,47,86,1000.0,DFW
